**TRANSFER LEARNING USING RESNET50**

In [2]:
#@title **Loading data from gdrive to memory**
# from google.colab import drive
# drive.mount('/content/drive')

# %cd /content/drive/MyDrive/ANNDL_Project

%cd ../../datasets

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/15O-hdeXPBICW_RJDpxFM_YoVERTpVMjK/ANNDL_Project


In [ ]:
!yes A | unzip data_splitted.zip -d data_splitted
!yes A | unzip data_cleaned_for_training.zip

In [ ]:
!pip install tensorflow==2.10.1

In [4]:
#@title **Imports**
import warnings
import logging
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
import numpy as np
import os
import random
import pandas as pd
import scipy
from datetime import datetime
from google.colab import files

In [5]:
#@title **Metadata and variables**

tfk = tf.keras
tfkl = tf.keras.layers

dir = "data_splitted"

input_shape = (96, 96, 3)
nclasses = 8
num_epochs_tl = 60
num_epochs_ft = 40
seed = 42

use_cleaned_data = True

In [6]:
#@title **Setting seed and/or suppressing warnings**
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Setting random seed for reproducibility
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [12]:
#@title **Image Dataset from directory (already resized to the shape needed by the pre-trained network)**
def get_image_dataset(dir, seed, width=96, height=96):
    
    train_dir = dir+"/train"
    # train_dir = "data_cleaned_for_training"
    test_dir = dir+"/test"
    val_dir = dir+"/val"
    train_ds = tfk.utils.image_dataset_from_directory(
        train_dir,
        label_mode='categorical',
        seed=seed,
        image_size=(224,224),
        batch_size = 32
    )
    val_ds = tfk.utils.image_dataset_from_directory(
        val_dir,
        label_mode='categorical',
        seed=seed,
        image_size=(224,224),
        batch_size = 32
    )
    test_ds = tfk.utils.image_dataset_from_directory(
        test_dir,
        label_mode='categorical',
        seed=seed,
        image_size=(224,224),
        batch_size = 32
    )
    
    return train_ds, val_ds, test_ds

In [7]:
#@title **Utility function to count number of samples in a directory**
def count_samples(directory):
  count = 0
  for root_dir, cur_dir, files in os.walk(directory):
      count += len(files)
  return count

In [8]:
#@title **Utility function to create folders and callbacks for training**
def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('trained_models/citte')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%m-%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(
                                                     filepath=ckpt_dir + '/cp-{val_accuracy:.2f}-{epoch:02d}.ckpt', # Checkpoint is saved with validation accuracy in the filename
                                                     monitor='val_accuracy', 
                                                     save_weights_only=True, # True to save only weights
                                                     save_best_only=True, # True to save only the best epoch 
                                                     initial_value_threshold=0.7
                                                     ) # Model is saved only if val_accuracy > initial_value_threshold

  callbacks.append(ckpt_callback)


  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  # By default shows losses and metrics for both training and validation
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
  callbacks.append(tb_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks, exp_dir

---

## **Transfer Learning**

In [9]:
#@title **Getting class weights to handle classes' sample imbalance**
def get_class_weights():

  num_images = count_samples(dir)
  elements = np.zeros(nclasses)
  weights = np.zeros(nclasses)

  for i in range(nclasses):
      elements[i] = count_samples("data_cleaned_for_training/Species"+str(i+1))
      weights[i] = (1 / elements[i]) * (num_images / float(nclasses))
      
  class_weight = {0: weights[0], 1: weights[1], 2: weights[2], 3: weights[3], 4: weights[4], 5: weights[5], 6: weights[6], 7: weights[7]}
  print("Samples count: ", elements)
  print("Class weights: ", weights)

  return class_weight
  

In [10]:
#@title **External Augmentation block**

img_augmentation = Sequential(
    [
        tfk.layers.RandomRotation(factor=1),
        tfk.layers.RandomZoom(width_factor=0.2, height_factor=0.2),
        tfk.layers.RandomFlip(),
        tfk.layers.RandomContrast(factor=0.3),
    ],
    name="img_augmentation",
)

In [11]:
#@title **Transfer Learning using pre-trained ResNet50 and augmentation block**

from tensorflow.keras.applications.resnet50 import preprocess_input


def build_ResNet50(input_shape = (224, 224, 3), nclasses = 8):
    model = tfk.Sequential()
    inputs = tfkl.Input(shape=(224, 224, 3))
    x = img_augmentation(inputs)
    # preprocessing of images needed by this type of pretrained network
    x = preprocess_input(x)
    res_model = tf.keras.applications.resnet50.ResNet50(
        input_tensor=x,
        include_top=False,
        weights='imagenet',
        pooling=None
    )
    # Freeze the pretrained weights
    res_model.trainable = False

    # Rebuild top
    dropout_rate = 0.5

    model = tf.keras.models.Sequential()
    model.add(res_model)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(8, activation='softmax'))

    # Compile
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=5e-4)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

tl_model = build_ResNet50()

94765736/94765736 [==============================] - 1s 0us/step


In [26]:
tl_model.summary(
    expand_nested=True,
    show_trainable=True
)

Model: "sequential_5"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  N          
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_5 (InputLayer)      [(None, 224, 224, 3)]     0         N          |
|                                                                          |
| img_augmentation (Sequentia  (None, 224, 224, 3)    0         N          |
| l)                                                                       |
||¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯||
|| random_rotation_3 (RandomRo  (None, 224, 224, 3)  0         N          ||
|| tation)                                                                ||
||                                                                        ||
|| random_zoom_3 (RandomZoom)  (None, 224, 224, 3)   0

In [41]:
#@title Transfer Learning training phase (with ImageDataGenerators)

callbacks, model_folder_dir = create_folders_and_callbacks(model_name='ResNet50_TL_CB')

train_gen, valid_gen, test_gen = get_image_dataset(dir=dir, seed=seed)

tl_history = tl_model.fit(
    x = train_gen,
    batch_size = 64,
    epochs = num_epochs_tl,
    callbacks = callbacks,
    validation_data = valid_gen,
    class_weight = get_class_weights()
    ).history

Found 2829 files belonging to 8 classes.
Found 351 files belonging to 8 classes.
Found 362 files belonging to 8 classes.
Samples count:  [141. 409. 388. 391. 404. 172. 429. 389.]
Class weights:  [3.14007092 1.08251834 1.14110825 1.13235294 1.09591584 2.57412791
 1.03205128 1.13817481]
Epoch 1/60
89/89 [==============================] - 825s 8s/step - loss: 3.2102 - accuracy: 0.1767 - val_loss: 1.6107 - val_accuracy: 0.4217
Epoch 2/60
89/89 [==============================] - 34s 378ms/step - loss: 2.7545 - accuracy: 0.2478 - val_loss: 1.3444 - val_accuracy: 0.5670
Epoch 3/60
89/89 [==============================] - 35s 392ms/step - loss: 2.5506 - accuracy: 0.3051 - val_loss: 1.2465 - val_accuracy: 0.5670
Epoch 4/60
89/89 [==============================] - 34s 373ms/step - loss: 2.3189 - accuracy: 0.3694 - val_loss: 1.1481 - val_accuracy: 0.6040
Epoch 5/60
89/89 [==============================] - 34s 376ms/step - loss: 2.1157 - accuracy: 0.4175 - val_loss: 1.1120 - val_accuracy: 0.6154
E

### Fine Tuning

In [ ]:
#@title Reloading the model after Transfer Learning (through checkpoints or the entire model)

# ft_model = tfk.models.load_model('saved_models/...')

ft_model = build_ResNet50()
ft_model.load_weights("trained_models/...")

In [ ]:
ft_model.summary(
    expand_nested=True,
    show_trainable=True
)

In [45]:
#@title **Unfreezing completely (or part of) the pretrained network and re-compiling**

'''
# It is also possible to partially unfreeze the pretrained network:

layers_to_unfreeze = 100
for i, layer in enumerate(ft_model.get_layer('resnet_50').layers[-layers_to_unfreeze:]):
    # BatchNormalization layers are not unfreezed because, if trainable, they lead to a performance decrease
    if not isinstance(layer, tfk.layers.BatchNormalization):
        layer.trainable = True

'''

ft_model.get_layer('resnet50').trainable = True


In [46]:
ft_model.summary(
    expand_nested=True,
    show_trainable=True
)

Model: "sequential_13"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  Y          
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_9 (InputLayer)      [(None, 224, 224, 3)]     0         Y          |
|                                                                          |
| img_augmentation (Sequentia  (None, 224, 224, 3)    0         Y          |
| l)                                                                       |
||¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯||
|| random_rotation_6 (RandomRo  (None, 224, 224, 3)  0         Y          ||
|| tation)                                                                ||
||                                                                        ||
|| random_zoom_6 (RandomZoom)  (None, 224, 224, 3)   

In [47]:
#@title Re-compile the model to apply changes
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

In [ ]:
#@title **Fine-Tuning the partially unfreezed model**

callbacks, model_folder_dir = create_folders_and_callbacks(model_name='ResNet50_FT_CB')

train_gen, valid_gen, test_gen = get_image_dataset(dir=dir, seed=seed)

ft_history = ft_model.fit(
    x = train_gen,
    batch_size = 64,
    epochs = num_epochs_ft,
    callbacks = callbacks,
    validation_data = valid_gen,
    class_weight = get_class_weights()
).history

Found 2829 files belonging to 8 classes.
Found 351 files belonging to 8 classes.
Found 362 files belonging to 8 classes.
Samples count:  [141. 409. 388. 391. 404. 172. 429. 389.]
Class weights:  [3.14007092 1.08251834 1.14110825 1.13235294 1.09591584 2.57412791
 1.03205128 1.13817481]
Epoch 1/40
89/89 [==============================] - 68s 635ms/step - loss: 1.5726 - accuracy: 0.5907 - val_loss: 1.0494 - val_accuracy: 0.6154
Epoch 2/40
89/89 [==============================] - 55s 611ms/step - loss: 1.2383 - accuracy: 0.6720 - val_loss: 1.0798 - val_accuracy: 0.6268
Epoch 3/40
89/89 [==============================] - 59s 654ms/step - loss: 1.1729 - accuracy: 0.6946 - val_loss: 0.7816 - val_accuracy: 0.7123
Epoch 4/40
89/89 [==============================] - 60s 671ms/step - loss: 1.1102 - accuracy: 0.7207 - val_loss: 0.6972 - val_accuracy: 0.7322
Epoch 5/40
89/89 [==============================] - 60s 664ms/step - loss: 1.0420 - accuracy: 0.7328 - val_loss: 0.6310 - val_accuracy: 0.7607

---

# **Save of models**


In [ ]:
model.save("saved_models/...")



---

# **Loading of models**

In [ ]:
#@title **Loading of an existing model (having only the weights)**
ft_model = build_ResNet50()
ft_model.load_weights('trained_models/...')

In [ ]:
#@title **Loading of an existing model (Complete)**
model = tf.keras.models.load_model("saved_models/...")

---
# **Model testing**


In [ ]:
#@title **Testing a model and re-saving**

train_gen, valid_gen, test_gen = get_image_dataset(dir=dir, seed=seed)
model_test_metrics = ft_model.evaluate(test_gen, return_dict=True)

# saving model in saved_models/modelname_accuracy
saved_model_name = 'ResNet50_' + str(model_test_metrics["accuracy"])
saved_model_dir = 'saved_models/' + saved_model_name
# ft_model.save(saved_model_dir)

Found 2829 files belonging to 8 classes.
Found 351 files belonging to 8 classes.
Found 362 files belonging to 8 classes.
12/12 [==============================] - 96s 5s/step - loss: 0.3683 - accuracy: 0.8812




---
# **Download of the model in .zip format**


In [ ]:
zipped_model = saved_model_dir + '/' + saved_model_name + '.zip'

In [ ]:
!zip -r {zipped_model} {saved_model_dir}

In [ ]:
files.download(zipped_model)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



---

